# Data preparation
## Main directories
**data** - 3 channels: 0 = celltracker (CT); 1 = LNP; 2 = brightfield (BF)
* 8 wells and two FOVs per well (one well excluded due to missing data)
* data for cells until they're tracking ends, i.e. they go too close to the edge of the FOV  
* well C03 is excluded as it has missing data - see below

**gfp** - 1 channel = GFP for the corresponding images in **data**

## Statistics to use when standardizing or normalizing the data
StatsRecorder below from http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.html
saved as 'statsrecorder.py'
* cell_stats - min, max, mean and sd (for each input channels)
* gfp values based on inner cut-outs (92 x 92 pixels)
* gfp_stats - min, max, mean, sd, gfp_thresh
* where gfp_thresh - the threshold for a positive gfp target (from control wells)

## Directories for modelling data
**images_train**, **images_test**, **gfp_train** and **gfp_test**
* models based on data for the first 20 time points to predict GFP at the final time point (t=72)
* data is a subset of the main directories for those cells that made it to the end (and only their first 20 time points)
* cells are shuffled into the training and test directories (all 20 images for each cell going into one or the other folder)

In [1]:
# stretching jupyter notebook view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import glob
import os, shutil
import datetime
import xml.etree.ElementTree as ET
import statsrecorder as sr

In [3]:
# misc functions
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [4]:
# overall image size
im_shape0 = 2154
im_shape1 = 2554

# path to images
image_folder = '/mnt/external-images-pvc/AZ/LNP/AssayPlate_Greiner_#781091/'

# offset = 96 -> tile size of 192 x 192 (divides nicely by two for when max pooling)
offset = 96
xydim = offset * 2

# warp matrix for BF registration
warp_matrix = np.load('warp.npy')

# CREATE MAIN DATA FOLDERS (with data across all 72 time points)
data_dir = '/scratch-shared/phil/LNP/LNP_data_09/data'
os.makedirs(data_dir)
gfp_dir = '/scratch-shared/phil/LNP/LNP_data_09/gfp'
os.makedirs(gfp_dir)

# well and FOV IDs
# well C03 excluded due to missing data
c_files = ['C04_F001', 'C04_F002', 'C05_F001', 'C05_F002',
          'E03_F001', 'E03_F002', 'E04_F001', 'E04_F002', 'E05_F001', 'E05_F002',
          'G03_F001', 'G03_F002', 'G04_F001', 'G04_F002', 'G05_F001', 'G05_F002']

# control files - no mRNA added to LNPs; used for determining threshold for a +ive GFP response
control_files = ['G03_F001', 'G03_F002', 'G04_F001', 'G04_F002', 'G05_F001', 'G05_F002']

# training and test folders for models based on t=1:20
t_stop = 20
images_train_dir = '/scratch-shared/phil/LNP/LNP_data_09/images_train'
os.makedirs(images_train_dir)
images_test_dir = '/scratch-shared/phil/LNP/LNP_data_09/images_test'
os.makedirs(images_test_dir)

# folders for the targets
gfp_train_dir = '/scratch-shared/phil/LNP/LNP_data_09/gfp_train'
os.makedirs(gfp_train_dir)
gfp_test_dir = '/scratch-shared/phil/LNP/LNP_data_09/gfp_test'
os.makedirs(gfp_test_dir)

### Add cell images to data_dir and gfp_dir

In [5]:
for c_file in c_files:
    s = c_file.split('_')
    f = glob.glob(image_folder + '/*' + s[0] + '*' + s[1] + '*C07*')
    files = sorted(f, key=natural_keys)
    # path to tracking_coords
    coords_file = 'tracking_coords/' + c_file + '.npy'
    coords = np.load(coords_file)
    for cell_nr in range(coords.shape[0]):
        for i, coord in enumerate(coords[cell_nr]):
            # if go outside window stop collecting data for cell
            if (coord[1] - offset < 0 or 
                coord[1] - offset + int(warp_matrix[1,2]) < 0 or
                coord[1] + offset > im_shape0 or 
                coord[1] + offset + int(warp_matrix[1,2]) > im_shape0 or
                coord[0] - offset < 0 or
                coord[0] - offset + int(warp_matrix[0,2]) < 0 or
                coord[0] + offset > im_shape1 or
                coord[0] + offset + int(warp_matrix[0,2]) > im_shape1):
                break
            # tp = time point in format '_T0001' to '_T0072'
            tp = files[i][-26:-20]
            # 0 = CT; 1 = LNP; 2 = BF; 3 = GFP
            f0 = glob.glob(image_folder + '/*' + s[0] + tp + '*' + s[1] + '*' + 'C07.tif')
            f1 = glob.glob(image_folder + '/*' + s[0] + tp + '*' + s[1] + '*' + 'C06.tif')
            f2 = glob.glob(image_folder + '/*' + s[0] + tp + '*' + s[1] + '*' + 'C09.tif')
            f3 = glob.glob(image_folder + '/*' + s[0] + tp + '*' + s[1] + '*' + 'C05.tif')
            im0 = cv2.imread(f0[0], -1)
            crop0 = im0[(coord[1] - offset):(coord[1] + offset), (coord[0] - offset):(coord[0] + offset)]
            im1 = cv2.imread(f1[0], -1)
            crop1 = im1[(coord[1] - offset):(coord[1] + offset), (coord[0] - offset):(coord[0] + offset)]
            im2 = cv2.imread(f2[0], -1)
            crop2 = im2[(coord[1] - offset + int(warp_matrix[1, 2])):
                        (coord[1] + offset + int(warp_matrix[1, 2])),
                        (coord[0] - offset + int(warp_matrix[0, 2])):
                        (coord[0] + offset + int(warp_matrix[0, 2]))]
            im_all = np.reshape(crop0, (xydim, xydim, 1))
            im_all = np.append(im_all, np.reshape(crop1, (xydim, xydim, 1)), axis=2)
            im_all = np.append(im_all, np.reshape(crop2, (xydim, xydim, 1)), axis=2)
            im3 = cv2.imread(f3[0], -1)
            crop3 = im3[(coord[1] - offset):(coord[1] + offset), (coord[0] - offset):(coord[0] + offset)]
            gfp = np.reshape(crop3, (xydim, xydim, 1))
            np.save(data_dir + '/' + c_file + tp + '_cell_'+ str(cell_nr) + '.npy', im_all)
            np.save(gfp_dir + '/' + c_file + tp + '_cell_'+ str(cell_nr) + '.npy', gfp)

In [6]:
train_split = 0.8 # 80% of data to training
train_y = []
train_name = []
test_y = []
test_name = []

# collecting gfp statistics from training data
# to allow for various standradization/normalization methods for the target when in regression mode
mystats = sr.StatsRecorder()
minmax = np.zeros(2)

lo = 48 # for centred cropping of gfp images
hi = 144

for c_file in c_files:
    # pulling out only those cells that made it to the end
    f_cell = glob.glob(gfp_dir + '/' + c_file + '*' + '_T0072' + '*')
    f_cell = sorted(f_cell, key = natural_keys)
    for i, f in enumerate(f_cell):
        im = np.load(f)
        im2 = im[lo:hi, lo:hi, 0] # centred crop
        ff = f.split('gfp/')[1]
        ff = ''.join(ff.split('_T0072'))
        if np.random.random() <= train_split:
            train_y = np.append(train_y, np.sum(im2))
            train_name.extend([ff])
            
            if np.sum(im2) < minmax[0]:
                minmax[0] = np.sum(im2)
            if np.sum(im2) > minmax[1]:
                minmax[1] = np.sum(im2)
            mystats.update(np.sum(im2))
        else:
            test_y = np.append(test_y, np.sum(im2))
            test_name.extend([ff])

# threshold for gfp computed across all time points for all control cells (for preprocessing)
gfp_thresh = 0
for t_file in control_files:
    files = glob.glob(gfp_dir + '/' + t_file + '*')
    for i, f in enumerate(files):
        im = np.load(f)
        im2 = im[lo:hi, lo:hi, 0] # centred crop
        if np.sum(im2) > gfp_thresh:
            gfp_thresh = np.sum(im2)

gfp_stats = np.zeros(5)
gfp_stats[0] = minmax[0]
gfp_stats[1] = minmax[1]
gfp_stats[2] = mystats.mean
gfp_stats[3] = mystats.std
gfp_stats[4] = gfp_thresh

np.save('/scratch-shared/phil/LNP/LNP_data_09/gfp_stats.npy', gfp_stats)

print('gfp stats:')
print(gfp_stats)

print('no. training cells =', len(train_y))
print('positives =', sum(train_y > gfp_thresh))
print('negatives =', sum(train_y <= gfp_thresh))

print('no. test cells =', len(test_y))
print('positives =', sum(test_y > gfp_thresh))
print('negatives =', sum(test_y <= gfp_thresh))

gfp stats:
[      0.         4484517.          487273.34219269  630943.22057337
  126950.        ]
no. training cells = 602
positives = 387
negatives = 215
no. test cells = 172
positives = 123
negatives = 49


In [7]:
# adding target data to gfp train and test directories
for name in train_name:
    s = name.split('_')
    f_cell = glob.glob(gfp_dir + '/' + s[0] + '_' + s[1] + '_T0072' + '_cell_' + s[3])
    im = np.load(f_cell[0])
    im2 = im[lo:hi, lo:hi, 0] # centred crop
    gfp = np.sum(im2)
    np.save(gfp_train_dir + '/' + s[0] + '_' + s[1] + '_T0072' + '_cell_' + s[3], gfp)

for name in test_name:
    s = name.split('_')
    f_cell = glob.glob(gfp_dir + '/' + s[0] + '_' + s[1] + '_T0072' + '_cell_' + s[3])
    im = np.load(f_cell[0])
    im2 = im[lo:hi, lo:hi, 0] # centred crop
    gfp = np.sum(im2)
    np.save(gfp_test_dir + '/' + s[0] + '_' + s[1] + '_T0072' + '_cell_' + s[3], gfp)

In [8]:
# preprocessing to get cell images in correct format for VGG16 base model
tpoints = '_T0001'
for i in range(2, 10):
    tpoints = np.append(tpoints, '_T000' + str(i))
for i in range(10, t_stop + 1):
    tpoints = np.append(tpoints, '_T00' + str(i))

# computing cell level stats for cell images (for preprocessing)
mystats = sr.StatsRecorder() # first channel
minmax = np.zeros((3,2))

# calculate min and max for training data (correcting for differences in dynamic ranges)
for name in train_name:
    for i, tp in enumerate(tpoints):
        s = name.split('_')
        f_cell = glob.glob(data_dir + '/' + s[0] + '_' + s[1] + tp + '_cell_' + s[3])
        im = np.load(f_cell[0])
        im_new = im.reshape(-1, im.shape[-1])
        for j in range(3):
            if np.min(im_new[:, j]) < minmax[j, 0]:
                minmax[j, 0] = np.min(im_new[:, j])
            if np.max(im_new[:, j]) > minmax[j, 1]:
                minmax[j, 1] = np.max(im_new[:, j])

# calculate mean once data is on the right scale for VGG16 input
for name in train_name:
    for i, tp in enumerate(tpoints):
        s = name.split('_')
        f_cell = glob.glob(data_dir + '/' + s[0] + '_' + s[1] + tp + '_cell_' + s[3])
        im = np.load(f_cell[0])
        im = im.astype('float32')
        im_new = im
        for j in range(3):
            im_new[:, :, j] = 255 * (im[:, :, j] - minmax[j, 0]) / minmax[j, 1]
        
        im_new2 = im_new.reshape(-1, im_new.shape[-1])
        mystats.update(im_new2)
        
cell_stats = np.zeros((3,4))
for j in range(3):
    cell_stats[j, 0] = minmax[j, 0]
    cell_stats[j, 1] = minmax[j, 1]

cell_stats[:, 2] = mystats.mean
cell_stats[:, 3] = mystats.std

np.save('/scratch-shared/phil/LNP/LNP_data_09/cell_stats.npy', cell_stats)

print('cell stats for CT:')
print(cell_stats[0])
print('cell stats for LNP:')
print(cell_stats[1])
print('cell stats for BF:')
print(cell_stats[2])

cell stats for CT:
[0.00000000e+00 3.21440000e+04 8.50557418e+00 5.95228701e+00]
cell stats for LNP:
[0.00000000e+00 1.07580000e+04 1.36255645e+00 1.19130562e+00]
cell stats for BF:
[   0.         4196.           76.11380371    7.76618654]


In [9]:
# adding cell image data to train and test directories
# and getting data into VGG16 input format (0-255 scale with training data channel means subtracted)
for name in train_name:
    # 5-fold cross-validation: random alocation to fold for each training cell
    fold = random.choice(['fold1', 'fold2', 'fold3', 'fold4', 'fold5'])
    
    for i, tp in enumerate(tpoints):
        s = name.split('_')
        f_cell = glob.glob(data_dir + '/' + s[0] + '_' + s[1] + tp + '_cell_' + s[3])
        im = np.load(f_cell[0])
        im = im.astype('float32')
        im_new = im
        for j in range(3):
            im_new[:, :, j] = 255 * ((im[:, :, j] - minmax[j, 0]) / minmax[j, 1]) - cell_stats[j, 2]
        
        np.save(images_train_dir + '/' + fold + '_' + s[0] + '_' + s[1] + tp + '_cell_' + s[3], im_new)

for name in test_name:
    for i, tp in enumerate(tpoints):
        s = name.split('_')
        f_cell = glob.glob(data_dir + '/' + s[0] + '_' + s[1] + tp + '_cell_' + s[3])
        im = np.load(f_cell[0])
        im = im.astype('float32')
        im_new = im
        for j in range(3):
            im_new[:, :, j] = 255 * (im[:, :, j] - minmax[j, 0]) / minmax[j, 1]
            # clipping any test images with a wider range than training images
            im_new[:, :, j] = np.clip(im_new[:, :, j], minmax[j, 0], minmax[j, 1])
            im_new[:, :, j] = im_new[:, :, j] - cell_stats[j, 2]
        
        np.save(images_test_dir + '/' + s[0] + '_' + s[1] + tp + '_cell_' + s[3], im_new)